In [1]:
'''Relegious Hate speech classifier using Natural Language processing(Naive bayes classifier)
By-Archit P. Hadge
B.Tech CSE VIT Pune(2018-2022)
'''
import GetOldTweets3 as got

In [2]:
#A function that returns list of tweets with given keyword of given quantity
def buildTestSet(keyword,quantity):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword).setMaxTweets(quantity)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    tweets=[{"text":tweet.text,"label":None,'link':tweet.permalink} for tweet in tweets]
    return tweets

#Function to get tweets of given keyword and provided label
def buildTrainSetSegment(keyword,quantity,label):
    trainSetSeg=[]
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword).setMaxTweets(quantity)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    for tweet in tweets:
        obj={}
        obj["topic"]=keyword
        obj["tweet_id"]=tweet.id
        obj["label"]=label
        obj["text"]=tweet.text
        obj["link"]=tweet.permalink
        trainSetSeg.append(obj)
    return trainSetSeg

import csv
#To create csv file containing tweets info
def buildCSV(filename,tweets):
    csvfile=open(filename,'a')
    linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
    for tweet in tweets:
        linewriter.writerow([tweet["topic"], tweet["tweet_id"], tweet["label"], tweet["text"],tweet["link"]])

#to read created csv file and return data from it
def readCSV(filename):
    tweets=[]
    csvfile=open(filename)
    lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
    for row in lineReader:
        #print(row[4])
        tweets.append({"topic":row[0],"tweet_id":row[1],"label":row[2],"text":row[3],"link":row[4]})
    return tweets

#Takes params as filename,list of keywords,total quantity and label and write it in file
def makeDataset(file,keywords,quantity,label):
    for i in keywords:
        buildCSV(file,buildTrainSetSegment(i,quantity//len(keywords),label))


l1=['jihad','kafir','allah','radical','kashmir','secularism','bharat tere tukde honge','love jihad','rohingya']
l1=['"'+x+'"' for x in l1]
#print(l)
l2=['table','business','goverment','defence','sales','finance','trade','music','history','art']
l2=['"'+x+'"' for x in l2]
makeDataset('demo.csv',l1,4000,'hate')
makeDataset('demo.csv',l2,3000,'normal')
print('done')


    
    

done


In [3]:
'''keywords=['jihad','kafir','allah','radical','kashmir','secularism','bharat tere tukde honge','love jihad','ram-mandir','jnu','liberals','islamophobia','hinduphobia','rohingya']
Train=[]
print(len(Train))
for x in keywords:
    Train=Train+buildTrainSetSegment(x,100,'positive')
keywords=['table','business','goverment','defence','sales','finance','trade','music','history','art']
for x in keywords:
    Train=Train+buildTrainSetSegment(x,100,'neutral')
print(len(Train))

Test=[]
'''

#reterive data from file 'demo.csv'
Train=readCSV('demo.csv')

In [4]:
#for preprocesssing and vectorising
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [5]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(Train)
#preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

In [ ]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)


In [ ]:
#classify hate speeches and prints link/text of the tweet
def classify(tweets):
    copy=tweets
    tweets = tweetProcessor.processTweets(tweets)
    predictions=[NBayesClassifier.classify(extract_features(tweet[0])) for tweet in tweets]
    for i in range(len(copy)):
        if predictions[i]=='normal':
            print(copy[i]['link'],"-->",copy[i]['text'])

p1=buildTestSet('market',10)
#print(p1)
classify(p1)
